In [4]:
#!/usr/bin/python
#-*- coding: utf-8 -*-

import pymongo as pym
import re
import pandas as pd

def insertion_in_labelised(candidates, sentiment, limit=400):
    '''Echelle de classification -1 Négatif 0 Neutre 1 Positif
    Critère de sélection d'un tweet : ne doit pas être un retweet, ne doit contenir le nom 
    que d'un seul candidat'''
    
    politician_list = ['fillon', 'macron', 'le pen', 'hamon', 'melenchon', 'mélenchon', 'bayrou', 'aignan']
    
    # le (?i) rend la regex insensible a la casse. Il ne faut pas laisser de liste vide
    # sinon l'algo va tirer n'importe quel tweet dans la base sans filtre particulier
    if sentiment=='neg':
        val = -1
        dico = {'fillon': '(?i)#penelopegate|#fillongate|#fillondemission',
                'macron': '(?i)#stopmacron||#lepionmacron|#macrongate|#hollandebis|#macrongirouette|#toutsaufmacron',
                'le pen': '(?i)#lepengate|#fngate',
                'hamon': '(?i)#bilalhamon|#plusjamaisps|#hamonpiègeacons|#hamonpiegeacons',
                'melenchon': '(?i)#placeholder_pour_melenchon_negatif',
                'mélenchon': '(?i)#placeholder_pour_melenchon_negatif'}
    elif sentiment=='pos':
        val = 1
        dico = {'fillon': '(?i)#stopchassealhomme|#fillonpresident|#projetfillon|#soutienfillon|#avecfillon',
                'macron': '(?i)#teammacron|#lafranceenmarche|#enmarcheausommet|#macronprésident|#macronpresident',
                'hamon': '(?i)#hamonpresident|#avechamon|#jevotepour',
                'le pen' : '(?i)#aunomdupeuple|#jechoisismarine|#vivemarine|#avecmarine',
                'melenchon': '(?i)#6erépublique|#6erepublique|#placeaupeuple|#CantStenchonTheMelenchon|#18mars2017',
                'mélenchon': '(?i)#6erépublique|#6erepublique|#placeaupeuple|#CantStenchonTheMelenchon|#18mars2017'}
    elif sentiment=='neu':
        # récupération des tweets des médias
        val = 0
        
        # Pour info : les principaux médias
        main_media_tags = ' via @libe| via @le_figaro| via @lobs| via @le_progres| via @lelab_e1| via @lepoint'\
        '| via @libe| via @franceinfo| via @20minutes| via @lavoixdun| via @yahooactufr| via @lemondefr'\
        '| via @bfmtv| via @lesechos| via @lexpress| via @france24| via @itele'
        
        # Les médias les plus probablement neutres
        media_tag = ' via @le_progres| via @lelab_e1'\
        '| via @franceinfo| via @20minutes| via @yahooactufr| via @lemondefr'\
        '| via @lesechos| via @lexpress| via @france24'
        dico = {candidate: media_tag for candidate in candidates}
    else:
        print('Choisir le sentiment parmi "neg", "pos" et "neu".')
        return
    
    # client = pym.MongoClient()
    client = pym.MongoClient('localhost', 27017)
    collection = client.tweet.tweet
    labelisedCollection = client.tweet.labelised
    
    count = 0
    
    for candidate in candidates:
        print(5*'-' + candidate + 5*'-')
        a_inserer = []

        # cherche un des éléments dans le dictionnaire de sentiments
        sentiment_regex = re.compile((dico[candidate]))

        # on exclut les tweets où plusieurs candidats sont cités
        stop_words = '|'.join([pol for pol in politician_list if pol!=candidate])
        print('Regex :', dico[candidate])
        print('Stop words :', stop_words)
        
        # filtrage des retweets en amont
        corpus = collection.find(
            filter={'$and': [{'t_text': {'$not': re.compile("^rt @")}},
                             {'t_text': sentiment_regex}, {'t_text': {'$not': re.compile(stop_words)}}]},
            projection={'_id':False, 't_id':1, 't_text':1}, limit=limit)

        for t in corpus: 
            a_inserer.append({'text': t['t_text'], 'sentiment': val, 'candidat': candidate,
                             't_id': t['t_id']})
        
        try:
            labelisedCollection.insert_many(a_inserer)
        except: # si vide
            pass
        
        print(len(a_inserer), 'insertions effectuees.')
        count += len(a_inserer)
        
    print(45 * '-')
    print(count, 'insertions au total.')
    
    drop_duplicates(client)
    client.close()
    
    return
    
def drop_duplicates(client):

    collection = client.tweet.labelised
    
    # retrait des doublons
    print('Retrait des doublons...')
    duplicates = []
    removepipe = [{"$group":{"_id":"$t_id", "dups":{"$push":"$_id"},
                             "count":{"$sum":1}}},{"$match":{"count":{"$gt":1}}}]
    count = 0
    try :
        for doc in collection.aggregate(removepipe):
            it = iter(doc['dups'])
            next(it)
            for id in it :
                count += 1
                duplicates.append(pym.DeleteOne({'_id':id}))
            if duplicates:    
                collection.bulk_write(duplicates)    
    except:
        pass
    
    print(count, 'doublons retirés.')
    print(collection.count(), 'tweets dans la base auto.')
    
    return

In [5]:
candidates = ['fillon', 'macron', 'le pen']
insertion_in_labelised(candidates, sentiment='pos', limit=700)

-----fillon-----
Regex : (?i)#stopchassealhomme|#fillonpresident|#projetfillon|#soutienfillon|#avecfillon
Stop words : macron|le pen|hamon|melenchon|mélenchon|bayrou|aignan
700 insertions effectuees.
-----macron-----
Regex : (?i)#teammacron|#lafranceenmarche|#enmarcheausommet|#macronprésident|#macronpresident
Stop words : fillon|le pen|hamon|melenchon|mélenchon|bayrou|aignan
700 insertions effectuees.
-----le pen-----
Regex : (?i)#aunomdupeuple|#jechoisismarine|#vivemarine|#avecmarine
Stop words : fillon|macron|hamon|melenchon|mélenchon|bayrou|aignan
535 insertions effectuees.
---------------------------------------------
1935 insertions au total.
Retrait des doublons...
1429 doublons retirés.
8893 tweets dans la base auto.


In [6]:
def mongo_to_df(collection, n_last_tweets=0, retweet=False):
    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(n_last_tweets)
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []
    
    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue

        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
    
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

In [18]:
client = pym.MongoClient('localhost', 27017)
col = client.tweet.labelised
col.count()

1723

In [19]:
df = mongo_to_df(col)

In [20]:
for t in df[df['sentiment']==0]['text']:
    print(t)

mais pourquoi une mesure qui concerne 15000 personnes ultra-riches fait-elle si peur en france ? https://t.co/bsm66axkrd via @jlmelenchon
présidentielle: c'est quoi les différences de programme entre benoît hamon et jean-luc... - https://t.co/hbcrm8gjik via @20minutes
la taxe robot de hamon soutenue par… bill gates https://t.co/7qmlk6vvyh via @libe
jadot met la pression à hamon en annonçant qu'il a récolté ses 500 parrainages https://t.co/g4jwytmsuc via @le_figaro poker menteur?
jérôme sainte-marie : mélenchon et hamon ne sont pas interchangeables https://t.co/9kddyxm4co via @le_figaro
coup de bol pour hamon : bill gates défend la taxe robots, elon musk le revenu universel https://t.co/pz9q3ez1ef via @lobs
france/monde | quand bill gates et elon musk défendent les idées de benoît hamon https://t.co/vpga5lkul2 via @le_progres
santé : benoît hamon veut modifier le financement de l'hôpital https://t.co/j3uudl1ajo via @lepoint
jean-luc mélenchon, stats facebook, twitter, classt des politic